## LSTM CNN

In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd

In [3]:
import pickle

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

Embeding

In [5]:
max_features = 20000
maxlen = 20
embedding_size = 128

Convolution

In [6]:
kernel_size = 5
filters = 64
pool_size = 4

LSTM

In [7]:
lstm_output_size = 128

Training

In [8]:
batch_size = 32
epochs = 20

In [9]:
def load_ds(path):
    vecfile = open(path,'r')
    return pickle.load(vecfile)

In [10]:
ds = load_ds('./data/ann/vectors_50_50.txt')
ds.head()

,category,tweet_sec
0,1,"[268, 459, 146, 2, 470, 4493]"
1,1,"[2264, 7, 9784, 3050]"
2,0,"[80, 172, 35, 1, 14, 39, 2265, 7, 9785, 2266, ..."
3,1,"[107, 41, 21, 471, 9, 659, 5, 216, 300, 5, 7, ..."
4,1,"[562, 106, 29, 563, 29, 422]"


In [11]:
print('Pad sequences (samples x time)')
tweet_sec = sequence.pad_sequences(ds['tweet_sec'], maxlen=maxlen)
print('tweet shape:', tweet_sec.shape)

Pad sequences (samples x time)
('tweet shape:', (15306, 20))


In [12]:
def build_arch():
    print('Build model...')
    model = Sequential()
    model.add(Embedding(max_features, embedding_size, input_length=maxlen))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(lstm_output_size,
                   activation='tanh', 
                   recurrent_activation='hard_sigmoid', 
                   dropout=0.2, 
                   recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [13]:
def cross_val(tweet_sec, target, nFolds):
    kFold = StratifiedKFold(n_splits=nFolds, shuffle=True)
    scores = []
    for train, test in kFold.split(tweet_sec,target):
        model = build_arch()
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(tweet_sec[train],
                  target[train],
                  batch_size=batch_size,
                  epochs=epochs,verbose=1,
                  validation_split=0.1)
        score, acc = model.evaluate(tweet_sec[test], target[test],batch_size=batch_size)
        scores.append(acc * 100)
    return scores   

In [14]:
#Training with the 100% of the data set
def train_model(tweet_sec,target):
    model = build_arch()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model.fit(tweet_sec, target,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1)        

In [15]:
results = cross_val(tweet_sec,ds['category'],10)

Build model...
Train on 12396 samples, validate on 1378 samples
Epoch 1/20
12396/12396 [==============================] - 21s - loss: 0.4119 - acc: 0.8097 - val_loss: 0.3000 - val_acc: 0.8716
Epoch 2/20
12396/12396 [==============================] - 19s - loss: 0.1948 - acc: 0.9269 - val_loss: 0.3346 - val_acc: 0.8701
Epoch 3/20
12396/12396 [==============================] - 20s - loss: 0.0933 - acc: 0.9668 - val_loss: 0.3896 - val_acc: 0.8614
Epoch 4/20
12396/12396 [==============================] - 20s - loss: 0.0442 - acc: 0.9853 - val_loss: 0.5077 - val_acc: 0.8520
Epoch 5/20
12396/12396 [==============================] - 20s - loss: 0.0230 - acc: 0.9935 - val_loss: 0.6606 - val_acc: 0.8411
Epoch 6/20
12396/12396 [==============================] - 20s - loss: 0.0152 - acc: 0.9956 - val_loss: 0.7086 - val_acc: 0.8345
Epoch 7/20
12396/12396 [==============================] - 20s - loss: 0.0112 - acc: 0.9958 - val_loss: 0.7774 - val_acc: 0.8454
Epoch 8/20
12396/12396 [================

12398/12398 [==============================] - 19s - loss: 0.1912 - acc: 0.9291 - val_loss: 0.2789 - val_acc: 0.8853
Epoch 3/20
12398/12398 [==============================] - 19s - loss: 0.0895 - acc: 0.9685 - val_loss: 0.3871 - val_acc: 0.8708
Epoch 4/20
12398/12398 [==============================] - 19s - loss: 0.0395 - acc: 0.9888 - val_loss: 0.4388 - val_acc: 0.8687
Epoch 5/20
12398/12398 [==============================] - 19s - loss: 0.0253 - acc: 0.9923 - val_loss: 0.5415 - val_acc: 0.8549
Epoch 6/20
12398/12398 [==============================] - 19s - loss: 0.0152 - acc: 0.9952 - val_loss: 0.6867 - val_acc: 0.8425
Epoch 7/20
12398/12398 [==============================] - 21s - loss: 0.0108 - acc: 0.9967 - val_loss: 0.7761 - val_acc: 0.8570
Epoch 8/20
12398/12398 [==============================] - 20s - loss: 0.0067 - acc: 0.9975 - val_loss: 0.9228 - val_acc: 0.8345
Epoch 9/20
12398/12398 [==============================] - 20s - loss: 0.0080 - acc: 0.9972 - val_loss: 0.7821 - val

12398/12398 [==============================] - 20s - loss: 0.0923 - acc: 0.9687 - val_loss: 0.3809 - val_acc: 0.8614
Epoch 4/20
12398/12398 [==============================] - 20s - loss: 0.0441 - acc: 0.9869 - val_loss: 0.5650 - val_acc: 0.8433
Epoch 5/20
12398/12398 [==============================] - 20s - loss: 0.0262 - acc: 0.9918 - val_loss: 0.5953 - val_acc: 0.8462
Epoch 6/20
12398/12398 [==============================] - 20s - loss: 0.0155 - acc: 0.9959 - val_loss: 0.7102 - val_acc: 0.8541
Epoch 7/20
12398/12398 [==============================] - 20s - loss: 0.0116 - acc: 0.9962 - val_loss: 0.7475 - val_acc: 0.8425
Epoch 8/20
12398/12398 [==============================] - 19s - loss: 0.0130 - acc: 0.9959 - val_loss: 0.8319 - val_acc: 0.8541
Epoch 9/20
12398/12398 [==============================] - 19s - loss: 0.0072 - acc: 0.9973 - val_loss: 0.9361 - val_acc: 0.8360
Epoch 10/20
12398/12398 [==============================] - 19s - loss: 0.0083 - acc: 0.9973 - val_loss: 0.9384 - va

In [16]:
print("Acc: %.2f (+/- %.2f)" %(np.mean(results), np.std(results)))

Acc: 84.08 (+/- 0.77)


In [17]:
for i in range (0,len(results)):
    print "Model %d, acc: %.2f " %(i+1,results[i])

Model 1, acc: 84.99 
Model 2, acc: 85.44 
Model 3, acc: 84.20 
Model 4, acc: 83.59 
Model 5, acc: 84.64 
Model 6, acc: 83.66 
Model 7, acc: 83.86 
Model 8, acc: 84.38 
Model 9, acc: 82.75 
Model 10, acc: 83.27 


In [16]:
trained_model = train_model(tweet_sec,ds['category'])

Build model...
Epoch 1/20
15306/15306 [==============================] - 25s - loss: 0.3825 - acc: 0.8264    
Epoch 2/20
15306/15306 [==============================] - 23s - loss: 0.1876 - acc: 0.9304    
Epoch 3/20
15306/15306 [==============================] - 23s - loss: 0.0944 - acc: 0.9677    
Epoch 4/20
15306/15306 [==============================] - 24s - loss: 0.0449 - acc: 0.9852    
Epoch 5/20
15306/15306 [==============================] - 24s - loss: 0.0233 - acc: 0.9925    
Epoch 6/20
15306/15306 [==============================] - 24s - loss: 0.0173 - acc: 0.9943    
Epoch 7/20
15306/15306 [==============================] - 24s - loss: 0.0117 - acc: 0.9963    
Epoch 8/20
15306/15306 [==============================] - 24s - loss: 0.0087 - acc: 0.9971    
Epoch 9/20
15306/15306 [==============================] - 24s - loss: 0.0082 - acc: 0.9971    
Epoch 10/20
15306/15306 [==============================] - 24s - loss: 0.0075 - acc: 0.9974    
Epoch 11/20
15306/15306 [=========

In [17]:
model_json = trained_model.model.to_json()
with open("./data/models/ccn_lstm/lstm_cnn.json",'w') as json_file:
    json_file.write(model_json)
trained_model.model.save_weights("./data/models/ccn_lstm/lstm_cnn_weights.h5")

In [18]:
trained_model.model.save("./data/models/ccn_lstm/lstm_cnn.h5")